# 위경도 데이터 정리

In [ ]:
import pandas as pd
import time
from tqdm import tqdm
import requests
import json

path = "/content/drive/MyDrive/multicampus/data/"

In [ ]:
def whether_gangnam(latitude, longitude):
    url = "https://apis.openapi.sk.com/tmap/geo/reversegeocoding?version=1"

    payload = {
        "appKey": "l7xx0ffb87c22bdb45ae8facaeeb7958ad36",
        "lon": longitude,
        "lat": latitude,
    }

    res = requests.get(url, params=payload)

    try:
        gu = json.loads(res.text)["addressInfo"]["gu_gun"]
    except:
        return "ERROR"

    if gu == "강남구":
        return True
    else:
        return False

In [ ]:
def reduce_df(df):
    df = df[(df["Latitude"]>37.454) & (df["Latitude"]<37.536) & (df["Longitude"]>127.018) & (df["Longitude"]<127.1266)]
    df = df.reset_index(drop=True)
    return df

## 버스정류장

In [ ]:
bus = pd.read_csv(path + "2022년_전국버스정류장 위치정보_데이터.csv")
bus = bus[bus["관리도시명"]=="서울"][["정류장명", "위도", "경도"]]
bus = bus.rename({"정류장명": "Name", "위도": "Latitude", "경도": "Longitude"}, axis=1)
bus = reduce_df(bus)
print(len(bus))
bus.head()

1097


,Name,Latitude,Longitude
0,성수동차고지,37.535793,127.055682
1,자양미소약국.자양골목시장,37.532750,127.077319
2,국민은행자양지점앞,37.531573,127.081413
3,자양아파트앞,37.532192,127.084608
4,자양동잠실대교북단,37.532955,127.084806


In [ ]:
for i in tqdm(range(len(bus))):
    bus.loc[i, "gangnam"] = whether_gangnam(bus.loc[i, "Latitude"], bus.loc[i, "Longitude"])

100%|██████████| 1097/1097 [12:06<00:00,  1.51it/s]


In [ ]:
bus = bus[bus["gangnam"]==True]
bus = bus.drop(["gangnam"], axis=1).reset_index(drop=True).reset_index()

print(len(bus))
print(bus.info())
bus.head()

538
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538 entries, 0 to 537
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      538 non-null    int64  
 1   Name       538 non-null    object 
 2   Latitude   538 non-null    float64
 3   Longitude  538 non-null    float64
dtypes: float64(2), int64(1), object(1)
memory usage: 16.9+ KB
None


,index,Name,Latitude,Longitude
0,0,뱅뱅사거리,37.486653,127.033097
1,1,래미안아파트.파이낸셜뉴스,37.491087,127.030987
2,2,신분당선강남역,37.494993,127.029112
3,3,지하철2호선강남역,37.500785,127.026370
4,4,논현역,37.506367,127.023728


In [ ]:
bus.to_csv(path + "bus.csv", index=False, encoding="utf-8")

## 택시승차대

In [ ]:
taxi = pd.read_csv(path + "서울특별시_택시승차대 현황_20210731.csv", encoding="cp949")
taxi = taxi[(taxi["전원투입"]=="OK") & (taxi["행정구"]=="강남구")][["위치명", "좌표(Poi_Y)", "좌표(Poi_X)"]]
taxi = taxi.rename({"위치명": "Name", "좌표(Poi_Y)": "Latitude", "좌표(Poi_X)": "Longitude"}, axis=1).reset_index(drop=True).reset_index()
taxi.loc[:, "Longitude"] = taxi["Longitude"].apply(lambda x: float(x.replace("/", ".")))

print(len(taxi))
print(taxi.info())
taxi.head()

14
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      14 non-null     int64  
 1   Name       14 non-null     object 
 2   Latitude   14 non-null     float64
 3   Longitude  14 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 576.0+ bytes
None


,index,Name,Latitude,Longitude
0,0,어반테일러 앞,37.508280,127.022290
1,1,가로수길 영동빌딩 (올리브영 가로수길 중앙점 앞),37.518136,127.022959
2,2,임페리얼팰리스호텔 앞,37.513700,127.035520
3,3,한국건설기술인협회별관 좌측(상아아파트 건너),37.516880,127.051360
4,4,삼원가든 앞,37.525000,127.033110


In [ ]:
taxi.to_csv(path + "taxi.csv", index=False, encoding="utf-8")

## 지하철역

In [ ]:
# 1 ~ 8호선
subway_1to8 = pd.read_csv(path + "서울교통공사_1_8호선 역사 좌표(위경도) 정보_20211231.csv", encoding="cp949")
subway_1to8["Name"] = subway_1to8.apply(lambda x: f"{x['호선']}호선 {x['역명']}역", axis=1)
subway_1to8 = subway_1to8[["Name", "위도", "경도"]]
subway_1to8 = subway_1to8.rename({"위도": "Latitude", "경도": "Longitude"}, axis=1)

print(len(subway_1to8))
print(subway_1to8.info())
subway_1to8.head()

276
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       276 non-null    object 
 1   Latitude   276 non-null    float64
 2   Longitude  276 non-null    float64
dtypes: float64(2), object(1)
memory usage: 6.6+ KB
None


,Name,Latitude,Longitude
0,1호선 서울역,37.553150,126.972533
1,1호선 시청역,37.563590,126.975407
2,1호선 종각역,37.570203,126.983116
3,1호선 종로3가역,37.570429,126.992095
4,1호선 종로5가역,37.570971,127.001900


In [ ]:
# 9호선
subway_9 = pd.read_csv(path + "국가철도공단_수도권9호선_역위치_20221117.csv").dropna()
subway_9["Name"] = subway_9.apply(lambda x: f"{x['선명']} {x['역명']}역", axis=1)
subway_9 = subway_9[["Name", "위도", "경도"]]
subway_9 = subway_9.rename({"위도": "Latitude", "경도": "Longitude"}, axis=1)

print(len(subway_9))
print(subway_9.info())
subway_9.head()

29
<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 0 to 28
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       29 non-null     object 
 1   Latitude   29 non-null     float64
 2   Longitude  29 non-null     float64
dtypes: float64(2), object(1)
memory usage: 928.0+ bytes
None


,Name,Latitude,Longitude
0,9호선 개화역,37.578608,126.798153
1,9호선 김포공항역,37.562434,126.801058
2,9호선 공항시장역,37.563726,126.810678
3,9호선 신방화역,37.567532,126.816601
4,9호선 마곡나루역,37.567336,126.829497


In [ ]:
subway = pd.concat([subway_1to8, subway_9], ignore_index=True)
subway = reduce_df(subway)
print(len(subway))
print(subway.info())
subway.head()

40
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Name       40 non-null     object 
 1   Latitude   40 non-null     float64
 2   Longitude  40 non-null     float64
dtypes: float64(2), object(1)
memory usage: 1.1+ KB
None


,Name,Latitude,Longitude
0,2호선 강변역,37.535161,127.094684
1,2호선 잠실나루역,37.520688,127.103836
2,2호선 잠실역,37.513305,127.100129
3,2호선 잠실새내역,37.520731,127.103738
4,2호선 종합운동장역,37.511008,127.073641


In [ ]:
for i in tqdm(range(len(subway))):
    subway.loc[i, "gangnam"] = whether_gangnam(subway.loc[i, "Latitude"], subway.loc[i, "Longitude"])

100%|██████████| 40/40 [00:26<00:00,  1.52it/s]


In [ ]:
subway = subway[subway["gangnam"]==True]
subway = subway.drop(["gangnam"], axis=1).reset_index(drop=True).reset_index()

print(len(subway))
print(subway.info())
subway.head()

22
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   index      22 non-null     int64  
 1   Name       22 non-null     object 
 2   Latitude   22 non-null     float64
 3   Longitude  22 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 832.0+ bytes
None


,index,Name,Latitude,Longitude
0,0,2호선 삼성역,37.508827,127.063203
1,1,2호선 선릉역,37.504257,127.048174
2,2,2호선 역삼역,37.500658,127.036430
3,3,3호선 압구정역,37.526169,127.028502
4,4,3호선 신사역,37.516438,127.020247


In [ ]:
subway.to_csv(path + "subway.csv", index=False, encoding="utf-8")

## 하나로

In [ ]:
import pandas as pd

path = "/content/drive/MyDrive/multicampus/data/"

bus = pd.read_csv(path + "bus.csv")
taxi = pd.read_csv(path + "taxi.csv")
subway = pd.read_csv(path + "subway.csv")

In [ ]:
bus["type"] = "bus"
taxi["type"] = "taxi"
subway["type"] = "subway"

cautionzone = pd.concat([bus, taxi], ignore_index=True)
cautionzone = pd.concat([cautionzone, subway], ignore_index=True)
cautionzone = cautionzone.drop(["index"], axis=1).reset_index()
cautionzone.to_csv(path + "cautionzone.csv", index=False, encoding="utf-8")

# 현 위치 기준으로 가까운 곳 탐지

In [ ]:
import pandas as pd

path = "/content/drive/MyDrive/multicampus/data/"

# 통합 csv
cautionzone = pd.read_csv(path + "cautionzone.csv")

In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from haversine import haversine
from tqdm import tqdm
import time

criteria = 200
place_list = []

# 현재 위치
latitude = 37.509812901728836
longitude = 127.06139307841329
here = (latitude, longitude)

start = time.time()
for i in range(len(cautionzone)):
    temp = (cautionzone.loc[i, "Latitude"], cautionzone.loc[i, "Longitude"])
    distance = haversine(here, temp, unit="m")

    if distance < criteria:
        print(cautionzone.loc[i, "type"], cautionzone.loc[i, "Name"])
        place_list.append({"type": cautionzone.loc[i, "type"], "latitude": cautionzone.loc[i, "Latitude"], "longitude": cautionzone.loc[i, "Longitude"]})
end = time.time()

print(f"\n{end-start:.5f} sec")

bus 삼성역7번출구
bus 무역센터
bus 무역센터
bus 한국무역센터.삼성역
bus 한국무역센터.삼성역
bus 삼성역그랜드인터컨티넨탈
bus 삼성역
bus 코엑스도심공항
bus 코엑스.에스엠타운
bus 삼성역
bus 삼성역7번출구
bus 삼성역7번출구
subway 2호선 삼성역

0.04406 sec


In [ ]:
place_list

[{'type': 'bus', 'latitude': 37.5104082, 'longitude': 127.0627189},
 {'type': 'bus', 'latitude': 37.5111539, 'longitude': 127.0615406},
 {'type': 'bus', 'latitude': 37.5093334, 'longitude': 127.0623552},
 {'type': 'bus', 'latitude': 37.508667, 'longitude': 127.0617005},
 {'type': 'bus', 'latitude': 37.5081065, 'longitude': 127.0616639},
 {'type': 'bus', 'latitude': 37.5084898, 'longitude': 127.0610138},
 {'type': 'bus', 'latitude': 37.5097547, 'longitude': 127.0631256},
 {'type': 'bus', 'latitude': 37.5095939, 'longitude': 127.059429},
 {'type': 'bus', 'latitude': 37.5104504, 'longitude': 127.0619369},
 {'type': 'bus', 'latitude': 37.5101888, 'longitude': 127.0626911},
 {'type': 'bus', 'latitude': 37.5095357, 'longitude': 127.0629275},
 {'type': 'bus', 'latitude': 37.5102475, 'longitude': 127.0626146},
 {'type': 'subway', 'latitude': 37.508827, 'longitude': 127.063203}]